## Análise da literatura

In [ ]:
from Bio import Entrez

In [ ]:
Entrez.email = "pg49837@uminho.pt"
handle = Entrez.einfo()

In [ ]:
result = Entrez.read(handle)
#handle.close()

In [ ]:
result["DbList"]

In [ ]:
handle = Entrez.einfo(db="pubmed")
record = Entrez.read(handle)
record["DbInfo"]["Description"]

In [ ]:
for field in record["DbInfo"]["FieldList"]:
    print("%(Name)s, %(FullName)s, %(Description)s" %field)

In [ ]:
handle = Entrez.esearch(db = "pubmed", term = "MTNR1B[title]", retmax ="40")
record = Entrez.read(handle)

In [ ]:
id_List = record["IdList"]

In [ ]:
record["Count"]

In [ ]:
handle = Entrez.esearch(db="nucleotide", term = "Homo sapiens[Orgn] AND MTNR1B[Gene]", idtype='acc')

In [ ]:
record = Entrez.read(handle)

In [ ]:
record["IdList"]
#record["IdList"][2] refere-se ao genoma humano

In [ ]:
from Bio import Medline
handle = Entrez.efetch(db = "pubmed", id= id_List, rettype = "medline", retmode = "text")
records = Medline.parse(handle)
print()

for record in records :
    print("Titulo:", record.get("TI", "Vazio"))                                     
    print()
    print("Abstract:", record.get("AB", "Vazio"))
    print("Autores:", record.get("AU", "Vazio"))
    print("Fonte:", record.get("SO", "Vazio"))
    print("="*140)
    print()

In [ ]:
handle = Entrez.esearch(db="nucleotide", term = "Homo sapiens[Orgn] AND MTNR1B[Gene]", idtype='acc')

In [ ]:
record = Entrez.read(handle)

In [ ]:
record["IdList"]

## Análise da sequência e das features presentes no NCBI

a) Aceder ao NCBI e guardar os ficheiros correspondentes aos genes escolhidos, podendo explorar possíveis variantes

In [ ]:
from Bio import Seq
from Bio import SeqIO

In [ ]:
record = SeqIO.read("MTNR1B.gb", "genbank")
record

In [ ]:
record.seq

In [ ]:
print(record.id)
print(record.name)
print(record.description)
print(record.dbxrefs)
print(len(record.annotations) )
print(record.annotations["source"] )
print(len(record.features))

In [ ]:
len(record)

b) Verificar as anotações dos genes de interesse

In [ ]:
record.annotations

c) Verificar e analisar a informação complementar fornecida pela lista de features e seus 
qualifiers

In [ ]:
# print(record.features)
for feat in record.features:
    print("-->" , feat)
print(f"Número de features: {len(record.features)}")

i) Localização e tipo

In [ ]:
from Bio.SeqFeature import SeqFeature, FeatureLocation
for feat in record.features:
    print(feat.type)
    print(feat.location)

ii ) Regiões codificantes

In [ ]:
featcds = [ ]
for i in range(len(record.features)):
    if record.features[i].type == "CDS":
        featcds.append(i)
for k in featcds:
    print (record.features[k].location)
for k in featcds:
    print (record.features[k].extract(record.seq))

In [ ]:
featcds

iii ) Proteína codificada e seu significado biológico (anotações do gene)

In [ ]:
from Bio.SeqFeature import SeqFeature, FeatureLocation
for feat in record.features:
    if feat.type == 'CDS':
        print(feat.qualifiers['product'])

In [ ]:
for feat in record.features:
    if feat.type == 'gene':
        print(feat.qualifiers["note"])

In [ ]:
for feat in record.features:
    if feat.type == "CDS":
        print(feat)

iv) Converter a FASTA

In [ ]:
from Bio import SeqIO
records = SeqIO.parse("MTNR1B.gb","genbank")
count = SeqIO.write(records, "MTNR1B.fasta","fasta")
print(f'Foi convertido {count} registo.')

## Análise de homologias por BLAST

##### Nesta parte também devemos ter que ir à base de dados NCBI para confirmar os resultados (rever aula 5)

As ferramentas de procura de homologias serão de especial relevo, nomeadamente para a 
procura de genes homólogos, bem como para a caracterização funcional dos genes 
selecionados. No primeiro caso, deverá configurar adequadamente as suas pesquisas ao nível 
da base de dados e desenvolver código para automatizar a decisão de existência de homologias 
significativas. No segundo caso, poderá analisar a lista de sequências homólogas e identificar 
padrões consistentes ao nível da função desempenhada por estas

In [ ]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW 
from Bio import SeqIO

In [ ]:
record = SeqIO.read(open("MTNR1B.fasta"), format= "fasta")
print(len(record.seq))

In [ ]:
result_handle = NCBIWWW.qblast("blastn","nt", record.seq)

In [ ]:
save_file = open("MTNR1B_b.xml","w")
save_file.write(result_handle.read())
save_file.close()
result_handle.close()

In [ ]:
result_handle = open("MTNR1B_b.xml")
blast_record = NCBIXML.parse(result_handle)
for br in blast_record:
    print(f"Database: {br.database}")
    print(f"Gap penalty:{br.gap_penalties}")

Número de alinhamentos do registo

Acession number, ID do hit, definicao

nºs de HSP (high scoring pair) do alinhamento, e-value, score, tamanho do alinhamento, numero de caracteres iguais

In [ ]:
print(len(br.alignments))

In [ ]:
for br_x in br.alignments:
    print(f"Acession number: {br_x.accession}")
    print(f"ID do hit: {br_x.hit_id}")
    print(f"Definição: {br_x.hit_def}")
    print(f"HSP: {br_x.hsps}")
    break
    #???

In [ ]:
for alignment in br.alignments:
    for hsp in alignment.hsps:
        print("        ***ALINHAMENTO***")
        print(f"E-value: {hsp.expect}")
        print(f"Score: {hsp.score}")
        print(f"Tamanho: {hsp.align_length}")
        print(f"Caracteres iguais: {len(hsp.match)}")
        print("Query " + hsp.query[100:200] + "...")
        print("Match " + hsp.match[100:200] + "...")
        print("Sbjct " + hsp.sbjct[100:200] + "...")
        print()

In [ ]:
from Bio import SearchIO

In [ ]:
blastq_result = SearchIO.read("MTNR1B_b.xml", "blast-xml")
print(blastq_result)

In [ ]:
#Primeiro hit, primeiro HSP
blast_hsp = blastq_result[0][0]    
print(blast_hsp)

In [ ]:
blast_hsp.query_range

In [ ]:
blast_hsp.evalue

In [ ]:
#Três primeiros hsp
blast_slice = blastq_result[:3]
print(blast_slice)

## Ferramentas de análise das propriedades da proteína

##### Ver bases de dados curadas, UniProt, SwissProt pelo Biopyhton, PDB, CDD

A base de dados UniProt permite aceder a toda a informação de um conjunto alargado de
proteínas. Os ficheiros da SwissProt podem ser tratados automaticamente pelo BioPython (ver 
exemplos na secção 10.1 do tutorial).
Note que os registos UniProt podem ter diferentes graus de revisão por parte dos curadores da 
base de dados, sendo nos casos em que o registo tenha sido manualmente curado uma fonte 
importante de informação.

In [ ]:
from Bio import ExPASy
from Bio import SeqIO

In [ ]:
#Transcrição

mtnr1b_mrna = record.seq.transcribe()
mtnr1b_mrna

In [ ]:
#Tradução
mtnr1b_prot = mtnr1b_mrna.translate()
mtnr1b_prot

In [ ]:
from collections import Counter
common_amino = Counter(mtnr1b_prot)
common_amino

In [ ]:
#Codões STOP
protein = mtnr1b_prot.split('*')
protein

In [ ]:
handle = ExPASy.get_sprot_raw("P49286")
seq_record = SeqIO.read(handle, "swiss")
id = seq_record.id
seq = seq_record.seq
tam = len(seq_record.seq)
name = seq_record.name
desc = seq_record.description
com = seq_record.annotations["comment"]
taxon = seq_record.annotations["taxonomy"]
organism = seq_record.annotations["organism"]
key = seq_record.annotations["keywords"]
print(f"ID {id} \n Sequência: {seq} \n Tamanho da sequência: {tam} bp")
print(f"Nome: {name} \n Descrição: {desc} \n Taxonomia: {taxon} \n Organismo: {organism} \n Keywords: {key}")

In [ ]:
seq_prot = seq_record.seq
result_handle = NCBIWWW.qblast('blastp','swissprot', seq_prot)
save_file = open("prot_blast_swiss.xml","w")
save_file.write(result_handle.read())
save_file.close()

blast_record = NCBIXML.parse(result_handle)

In [ ]:
#Protein BLAST swissprot
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastp", "swissprot", seq_record.seq)

In [ ]:
from Bio import SearchIO
blast_records = SearchIO.read(result_handle, "blast-xml")

In [ ]:
print(blast_records[:])

In [ ]:
for br in blast_records:
    print(f'Sequence ID: {br.id}')
    print(f'Description: {br.description}')
    print(f'E-value: {br[0].evalue}')
    print(f'Bit Score: {br[0].bitscore}')
    print(f'Alignment:\n{br[0].aln}')
    print()

In [ ]:
print(blast_records[:])

In [ ]:
protein_seq = SeqIO.read("protein_seq.fasta","fasta")

In [ ]:

protein_seq.seq

Por outro lado, a base de dados PDB contém informação sobre a estrutura das proteínas. Poderá 
efetuar pesquisas nesta base de dados no sentido de identificar proteínas de interesse que 
estejam presentes nesta base de dados. As proteínas de interesse podem ser analisadas 
identificando zonas de possível ligação de compostos que possam regular o seu funcionamento.
Complementarmente, foram estudadas ferramentas que permitem inferir características da 
proteína com base na sua sequência, como sejam a sua localização celular, a existência de 
domínios transmembranares ou alterações pós-tradução relevantes. Todas estas ferramentas 
permitem dar pistas sobre as proteínas de interesse.

Foram ainda abordadas bases de dados de domínios de proteínas, das quais se destaca a NCBI 
CDD (conserved domain database) do NCBI. Esta base de dados, ou outras similares, pode ser 
usada para confirmar a anotação de proteínas de interesse, sendo de particular utilidade quando 
subsistem dúvidas sobre a anotação, quer esta provenha da anotação original, quer provenha 
de resultados de homologia (e.g. BLAST). Por outro lado, permite a análise dos domínios 
presentes na proteína, de forma a poder caracterizar potenciais pontos de ligação de compostos 
e outras proteínas que possam inibir o funcionamento da proteína

In [ ]:
from Bio.PDB.PDBParser import PDBParser

In [ ]:
#Protein BLAST cdd
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastp", "CDD", seq_record.seq)

In [ ]:
blast_records = SearchIO.read(result_handle, "blast-xml")

In [ ]:
print(blast_records[:])

In [ ]:
for br in blast_records:
    print(f'Sequence ID: {br.id}')
    print(f'Description: {br.description}')
    print(f'E-value: {br[0].evalue}')
    print(f'Bit Score: {br[0].bitscore}')
    print(f'Alignment:\n{br[0].aln}')
    print()

In [ ]:
prots = []
for p in protein:
    if len(p) > 20:
        prots.append(p)
prots

In [ ]:
with open ("protein_seq.fasta","w") as file:
    file.write(f">Protein: \n {prots[:]}")

In [ ]:
from Bio import SeqIO
protein_seq = SeqIO.read("protein_seq.fasta","fasta")

In [ ]:
protein_seq.seq

In [ ]:
#Protein BLAST
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastp", "pdb", protein_seq)

In [ ]:
from Bio import SearchIO
blast_records = SearchIO.read(result_handle, "blast-xml")

In [ ]:
print(blast_records[:])

## Alinhamentos múltiplos e filogenia

selecionar-se a sequência de interesse do organismo e um conjunto de
sequências homólogas (e.g. provenientes de um processo de BLAST) de organismos
selecionados, realizar o seu alinhamento múltiplo e complementarmente determinar a árvore
filogenética correspondente

In [ ]:
from Bio import Phylo
from Bio import AlignIO
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Align import AlignInfo

In [ ]:
result_blast = open("MTNR1B_b.xml")
blast_records = NCBIXML.read(result_blast)
for alignment in blast_records.alignments:
    for hsp in alignment.hsps:
        print(">", alignment.title, "\n", hsp.query[0:45])
        print()

In [ ]:
#guardou se manualmente o alinhamento anterior no ficheiro mtnr1b_blast.txt
alignments = AlignIO.parse("MTNR1B_BLAST.txt",format = "fasta")
for alignment in alignments:
    print(alignment)

In [ ]:
AlignIO.write(alignment, "align_results_mtnr1b", "fasta")

In [ ]:
summary_align = AlignInfo.SummaryInfo(alignment)
consensus = summary_align.dumb_consensus()
consensus

In [ ]:
alignment = AlignIO.read(open("align_results_mtnr1b.sth"),"stockholm")
print(alignment)

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio import AlignIO
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
calculator = DistanceCalculator('blosum62')
dm = calculator.get_distance(alignment)
print(dm)

In [ ]:
constructor = DistanceTreeConstructor()
upgmatree = constructor.upgma(dm)
print(upgmatree)

In [ ]:
seq1 = "MHQAIFIYQIGYPLKSGYIQSIRSPEYDNW"
seq2 = "MH--IFIYQIGYALKSGYIQSIRSPEY-NW"

seqr1 = SeqRecord(Seq(seq1),id="seq1")
seqr2 = SeqRecord(Seq(seq2),id="seq2")
alin = MultipleSeqAlignment([seqr1, seqr2])
print (alin)


In [ ]:
align1 = MultipleSeqAlignment([SeqRecord(Seq("ACTGCTAGC"), id="A"), 
    SeqRecord(Seq("ACT-CTAGC"), id="B"), 
    SeqRecord(Seq("ACTGCTAGD"), id="C"), ]) 
align2 = MultipleSeqAlignment([ SeqRecord(Seq("TCAGC-AG"), id="D"), 
    SeqRecord(Seq("ACAGCTAG"), id="E"), 
    SeqRecord(Seq("TCAGCTAG"), id="F"), ])

my_alignments = [align1, align2]

AlignIO.write(my_alignments, "my_example.phy", "phylip") 
AlignIO.write(my_alignments, "my_exampl.sth", "stockholm")
AlignIO.write(my_alignments, "my_examp.faa", "fasta")

In [ ]:
alignment = AlignIO.read("my_exampl.sth", "stockholm")
print (alignment)
 
print ("tamanho alinhamento %i" % alignment.get_alignment_length() )
 
for record in alignment:
    print ("%s - %s" % (record.seq, record.id) )
for record in alignment:
    if record.dbxrefs: 
        print (record.id, record.dbxrefs)

In [ ]:
alignment = AlignIO.read("MTNR1B.faa", "fasta")
print (alignment )

print ("tam. alinhamento %i" % alignment.get_alignment_length() )
 
for record in alignment:
    print ("%s - %s" % (record.seq, record.id) )

Multiplos alinhamentos (formato phylip)

In [ ]:
alignments = AlignIO.parse("MTNR1B.phy", "phylip") 
for alignment in alignments: 
    print (alignment)

In [ ]:
lalignments = list(AlignIO.parse("MTNR1B.phy", "phylip")) 
print (lalignments[-1])
print (lalignments[0])

arvore filogeni

In [ ]:
tree = Phylo.read("example.dnd", "newick")

In [ ]:
Phylo.convert("int_node_labels.nwk", "newick", "tree.xml", "phyloxml")